# Rat RNN

### Load Dependencies

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import tensorflow as tf

from unityagents import UnityEnvironment

%matplotlib inline

### Hyperparameters

In [ ]:
learning_rate = 1e-4
hidden_units = 64
train_steps = 2000
ep_length = 200
sim_steps = 50000
batch_size = 8

model_path = 'rat_model'
data_path = 'rat_data'
load_model = True

## Generate Data

### Start Environment

In [ ]:
env_name = "rat" # Name of the Unity environment binary to launch
env = UnityEnvironment(file_name=env_name, worker_id=1)

# Examine environment parameters
print(str(env))

# Set the default brain to work with
default_brain = env.brain_names[0]

### Collect States

In [ ]:
x,z,ep = [], [], []
input_x = []
a = env.reset()
for i in range(sim_steps):
    a = env.step()[default_brain]
    x.append(a.states[0][0])
    z.append(a.states[0][1])
    input_x.append(a.states[0][2:-1])
    ep.append(a.states[0][-1])

In [ ]:
x = np.array(x)
z = np.array(z)
input_x = np.array(input_x)
input_y = np.stack((x, z), axis=1)

### Save Training Data

In [ ]:
if not os.path.exists(data_path):
    os.makedirs(data_path)

In [ ]:
save_data = {"x": input_x, "y": input_y}
pickle.dump(save_data, open(data_path+"/data.p", "wb"))

## Load Training Data

In [ ]:
save_data = pickle.load(open(data_path+"/data.p", "rb"))
input_x = save_data["x"]
input_y = save_data["y"]
x = input_y[:, 0]
z = input_y[:, 1]

### Visualize Rat Trajectory

In [ ]:
def create_colors(length, segments):
    c_list = []
    for i in range(segments):
        color = np.zeros([length//segments, 4])
        color[:, 3] = 0.5
        color[:, 0:3] = np.random.uniform(0, 1, size=[1, 3])
        c_list.append(color)
    c_list = np.reshape(c_list, [length, 4])
    return c_list

In [ ]:
plt.figure(figsize=(15, 15))
plt.scatter(x, z, s=1, c=create_colors(sim_steps, sim_steps//ep_length))
plt.ylim(-4.5, 4.5)
plt.xlim(-4.5, 4.5)

## Rat RNN

### Define Network

In [ ]:
class RatRNN(object):
    def __init__(self, x_size, y_size, h_size, lr):
        self.x = tf.placeholder(shape=[None, x_size], dtype=tf.float32)
        self.train_length = tf.placeholder(shape=[1], dtype=tf.int32)
        self.batch_size = tf.placeholder(shape=[1], dtype=tf.int32)

        self.rnn_in = tf.reshape(self.x, shape=[self.batch_size[0], self.train_length[0], x_size])

        cell = tf.contrib.rnn.BasicLSTMCell(h_size, state_is_tuple=True)
        self.state_in = cell.zero_state(self.batch_size[0], tf.float32)

        self.rnn, self.rnn_state = tf.nn.dynamic_rnn(inputs=self.rnn_in, cell=cell, dtype=tf.float32,
                                                     initial_state=self.state_in)

        self.output = tf.reshape(self.rnn, shape=[-1, h_size])
        
        self.y_pred = tf.layers.dense(self.output, y_size, activation=None)

        self.y = tf.placeholder(shape=[None, y_size], dtype=tf.float32)

        l2_loss_a = tf.reduce_sum(tf.nn.l2_loss(tf.trainable_variables()[0]))
        l2_loss_b = tf.reduce_sum(tf.nn.l2_loss(tf.trainable_variables()[1]))
        self.l2_loss = l2_loss_a + l2_loss_b
        self.regress_loss = tf.reduce_sum(tf.squared_difference(self.y, self.y_pred))
        self.loss = self.regress_loss + self.l2_loss

        optimizer = tf.train.AdamOptimizer(learning_rate=lr)
        self.update = optimizer.minimize(self.loss)

### Train Network

In [ ]:
if not os.path.exists(model_path):
    os.makedirs(model_path)

tf.reset_default_graph()
rat_rnn = RatRNN(3, 2, hidden_units, learning_rate)
saver = tf.train.Saver()
sess = tf.InteractiveSession()  

if load_model:
    ckpt = tf.train.get_checkpoint_state(model_path)
    if ckpt == None:
      print('The model {0} could not be found. Make sure you specified the right '
        '--run-path'.format(model_path))
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    losses = []
    init = tf.global_variables_initializer()
    sess.run(init)
    for i in range(train_steps):
        pointer = np.random.randint(0, sim_steps/ep_length / batch_size)
        batch_x = input_x[pointer*ep_length*batch_size:(pointer+1)*ep_length*batch_size]
        batch_y = input_y[pointer*ep_length*batch_size:(pointer+1)*ep_length*batch_size]
        feed_dict = {rat_rnn.x: np.reshape(batch_x, [-1,3]), rat_rnn.y: np.reshape(batch_y, [-1, 2]), 
                     rat_rnn.train_length: [ep_length], rat_rnn.batch_size: [batch_size]}
        loss, _ = sess.run([rat_rnn.loss, rat_rnn.update], feed_dict=feed_dict)
        if i % 100 == 0 and i != 0:
            print("Loss: {}".format(loss))
        losses.append(loss)
    plt.plot(losses)
    saver.save(sess, model_path + '/model.cptk')

### Compare Trajectories

In [ ]:
plt.figure(figsize=(15,10))
for i in range(15):
    pointer = i
    batch_x = input_x[pointer*ep_length:(pointer+1)*ep_length]
    batch_y = input_y[pointer*ep_length:(pointer+1)*ep_length]
    feed_dict = {rat_rnn.x: np.reshape(batch_x, [-1,3]), 
                 rat_rnn.train_length: [ep_length], rat_rnn.batch_size: [1]}
    yp = sess.run(rat_rnn.y_pred, feed_dict=feed_dict)
    
    # Real - Green | Predicted - Blue
    plt.subplot(3, 5, 1 + i)
    plt.scatter(batch_y[:,0], batch_y[:, 1], s=1, c=[0,1,0,1])
    plt.scatter(yp[:, 0], yp[:, 1], s=1, c=[0,0,1,1])
    plt.ylim(-4.5, 4.5)
    plt.xlim(-4.5, 4.5)
plt.show()

### Close TensorFlow Session & Environment

In [ ]:
sess.close()
env.close()